## 1. Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 2. Setup Environment

In [2]:
# Chuyển đến thư mục project
%cd "/content/drive/MyDrive/Đồ án Python/coffee_project"

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', module='sklearn')

# Import libraries
import sys
import os
sys.path.append(os.getcwd())

from src.models.trainer import ModelTrainer, TrainingConfig
from src.models.evaluator import ClusteringEvaluator
from src.models.tuning import HyperparameterTuner, TuningConfig
import pandas as pd
import logging

# Setup logger
logging.basicConfig(level=logging.INFO, format='%(message)s')

print("✅ Environment setup complete!")

/content/drive/MyDrive/Đồ án Python/coffee_project
✅ Environment setup complete!


## 3. Hyperparameter Tuning (Advanced)
Grid search tất cả models để tìm best hyperparameters

In [3]:
print("="*80)
print("K-MEANS HYPERPARAMETER TUNING")
print("="*80)

# 1. Thiết lập Grid cho K-Means
# n_clusters: Số lượng cụm muốn thử (quan trọng nhất)
# init: Phương pháp khởi tạo tâm cụm
# n_init: Số lần chạy thuật toán với các tâm ngẫu nhiên khác nhau
kmeans_grid = {
    "n_clusters": [3, 4, 5, 6, 7, 8, 9, 10],
    "init": ["k-means++"],
    "n_init": [10, 20],
    "max_iter": [300]
}

print("Grid parameters:")
for key, values in kmeans_grid.items():
    print(f"  {key}: {values}")

total = 1
for v in kmeans_grid.values():
    total *= len(v)
print(f"\nTotal combinations: {total}")

# 2. Khởi tạo Tuner
evaluator = ClusteringEvaluator()
tuning_config = TuningConfig(
    data_path="data/processed/encoded_data.csv",
    results_path="results/kmeans_tuning.csv",  # Đổi tên file lưu
    metric_selection="silhouette"              # Metric chính để chọn model
)
tuner = HyperparameterTuner(config=tuning_config, evaluator=evaluator)

# 3. Chạy Tuning
# Lưu ý: "kmeans" phải khớp với tên model trong ModelTrainer
tuner.run_grid_search("kmeans", kmeans_grid)

# 4. Lưu kết quả tổng hợp
tuner.save_results()

# 5. Lấy kết quả tốt nhất và Train lại
df_results = tuner.get_summary()

if len(df_results) > 0:
    best = df_results.iloc[0]

    print("\n" + "="*80)
    print("🏆 BEST K-MEANS CONFIG")
    print("="*80)
    # Hiển thị các tham số của K-Means
    print(f"  n_clusters             : {int(best['n_clusters'])}")
    print(f"  init                   : {best['init']}")
    print(f"  n_init                 : {best['n_init']}")

    print(f"\n  Silhouette Score       : {best['silhouette']:.4f}")
    print(f"  Calinski-Harabasz      : {best['calinski_harabasz']:.2f}")
    print(f"  Davies-Bouldin Index   : {best['davies_bouldin']:.4f}")

    # Train và lưu model tốt nhất
    print("\n🔄 Training best model...")

    # Chuẩn bị params (n_clusters thường được tách riêng trong logic của trainer)
    best_params = {
        "init": best['init'],
        "n_init": int(best['n_init']),
        "max_iter": int(best['max_iter'])
    }

    config = TrainingConfig(
        data_path="data/processed/encoded_data.csv",
        model_type="kmeans",
        n_clusters=int(best['n_clusters']), # Quan trọng: KMeans cần n_clusters
        model_params=best_params,
        model_path="results/kmeans_best.pkl"
    )

    trainer = ModelTrainer(config=config, evaluator=evaluator)
    trainer.load_data()
    trainer.train_model()
    trainer.save_model()
    trainer.save_labels("results/kmeans_best_labels.csv")

    print("✅ Best K-Means model saved!")
    print("   Model: results/kmeans_best.pkl")
    print("   Labels: results/kmeans_best_labels.csv")

    # Show top 5 configs
    print("\n" + "="*80)
    print("📊 TOP 5 CONFIGS")
    print("="*80)
    # Chỉ chọn các cột có ý nghĩa với K-Means
    cols = ['n_clusters', 'init', 'n_init',
            'silhouette', 'calinski_harabasz', 'davies_bouldin']
    # Lọc các cột thực sự tồn tại trong df kết quả (đề phòng lỗi key)
    cols = [c for c in cols if c in df_results.columns]
    print(df_results[cols].head(5).to_string(index=False))

else:
    print("⚠️ No valid results found")

📂 Loading data for tuning from data/processed/encoded_data.csv...


K-MEANS HYPERPARAMETER TUNING
Grid parameters:
  n_clusters: [3, 4, 5, 6, 7, 8, 9, 10]
  init: ['k-means++']
  n_init: [10, 20]
  max_iter: [300]

Total combinations: 16


  ✓ Data loaded: (3685, 58)

🚀 Starting Grid Search for: KMEANS
  Total combinations to test: 16
  [1/16] KMEANS_n_clusters3_initk-means++_n_init10_max_iter300: Sil=0.2544, CH=1269.22, DB=1.5338, Clusters=3
  [2/16] KMEANS_n_clusters3_initk-means++_n_init20_max_iter300: Sil=0.2544, CH=1269.22, DB=1.5338, Clusters=3
  [3/16] KMEANS_n_clusters4_initk-means++_n_init10_max_iter300: Sil=0.2355, CH=1030.48, DB=1.8631, Clusters=4
  [4/16] KMEANS_n_clusters4_initk-means++_n_init20_max_iter300: Sil=0.2355, CH=1030.48, DB=1.8631, Clusters=4
  [5/16] KMEANS_n_clusters5_initk-means++_n_init10_max_iter300: Sil=0.2352, CH=894.63, DB=1.7419, Clusters=5
  [6/16] KMEANS_n_clusters5_initk-means++_n_init20_max_iter300: Sil=0.2352, CH=894.63, DB=1.7419, Clusters=5
  [7/16] KMEANS_n_clusters6_initk-means++_n_init10_max_iter300: Sil=0.1937, CH=820.04, DB=1.8492, Clusters=6
  [8/16] KMEANS_n_clusters6_initk-means++_n_init20_max_iter300: Sil=0.1938, CH=820.06, DB=1.8499, Clusters=6
  [9/16] KMEANS_n_clusters7

model_type  n_clusters  silhouette  calinski_harabasz  davies_bouldin
    kmeans           3    0.254433        1269.217570        1.533831
    kmeans           3    0.254433        1269.217570        1.533831
    kmeans           4    0.235535        1030.477913        1.863101

🏆 BEST K-MEANS CONFIG
  n_clusters             : 3
  init                   : k-means++
  n_init                 : 10

  Silhouette Score       : 0.2544
  Calinski-Harabasz      : 1269.22
  Davies-Bouldin Index   : 1.5338

🔄 Training best model...


  ✓ Model trained successfully
💾 Model saved: results/kmeans_best.pkl
💾 Labels saved: results/kmeans_best_labels.csv


✅ Best K-Means model saved!
   Model: results/kmeans_best.pkl
   Labels: results/kmeans_best_labels.csv

📊 TOP 5 CONFIGS
 n_clusters      init  n_init  silhouette  calinski_harabasz  davies_bouldin
          3 k-means++      10    0.254433        1269.217570        1.533831
          3 k-means++      20    0.254433        1269.217570        1.533831
          4 k-means++      10    0.235535        1030.477913        1.863101
          4 k-means++      20    0.235535        1030.477913        1.863101
          5 k-means++      10    0.235202         894.634671        1.741909


In [4]:
print("="*80)
print("K-MEANS HYPERPARAMETER TUNING")
print("="*80)

# 1. Thiết lập Grid cho K-Means
# n_clusters: Số lượng cụm muốn thử (quan trọng nhất)
# init: Phương pháp khởi tạo tâm cụm
# n_init: Số lần chạy thuật toán với các tâm ngẫu nhiên khác nhau
kmeans_grid = {
    "n_clusters": [3, 4, 5, 6, 7, 8, 9, 10],
    "init": ["k-means++"],
    "n_init": [10, 20],
    "max_iter": [300]
}

print("Grid parameters:")
for key, values in kmeans_grid.items():
    print(f"  {key}: {values}")

total = 1
for v in kmeans_grid.values():
    total *= len(v)
print(f"\nTotal combinations: {total}")

# 2. Khởi tạo Tuner
evaluator = ClusteringEvaluator()
tuning_config = TuningConfig(
    data_path="data/processed/encoded_data.csv",
    results_path="results/kmeans_composite_tuning.csv",
    metric_selection="composite",
    silhouette_weight=0.4,
    calinski_weight=0.3,
    davies_weight=0.3
)
tuner = HyperparameterTuner(config=tuning_config, evaluator=evaluator)

# 3. Chạy Tuning
# Lưu ý: "kmeans" phải khớp với tên model trong ModelTrainer
tuner.run_grid_search("kmeans", kmeans_grid)

# 4. Lưu kết quả tổng hợp
tuner.save_results()

# 5. Lấy kết quả tốt nhất và Train lại
df_results = tuner.get_summary()

if len(df_results) > 0:
    best = df_results.iloc[0]

    print("\n" + "="*80)
    print("🏆 BEST K-MEANS CONFIG")
    print("="*80)
    # Hiển thị các tham số của K-Means
    print(f"  n_clusters             : {int(best['n_clusters'])}")
    print(f"  init                   : {best['init']}")
    print(f"  n_init                 : {best['n_init']}")

    print(f"\n  Silhouette Score       : {best['silhouette']:.4f}")
    print(f"  Calinski-Harabasz      : {best['calinski_harabasz']:.2f}")
    print(f"  Davies-Bouldin Index   : {best['davies_bouldin']:.4f}")

    # Train và lưu model tốt nhất
    print("\n🔄 Training best model...")

    # Chuẩn bị params (n_clusters thường được tách riêng trong logic của trainer)
    best_params = {
        "init": best['init'],
        "n_init": int(best['n_init']),
        "max_iter": int(best['max_iter'])
    }

    config = TrainingConfig(
        data_path="data/processed/encoded_data.csv",
        model_type="kmeans",
        n_clusters=int(best['n_clusters']), # Quan trọng: KMeans cần n_clusters
        model_params=best_params,
        model_path="results/kmeans_composite_best.pkl"
    )

    trainer = ModelTrainer(config=config, evaluator=evaluator)
    trainer.load_data()
    trainer.train_model()
    trainer.save_model()
    trainer.save_labels("results/kmeans_best_composite_labels.csv")

    print("✅ Best K-Means model saved!")
    print("   Model: results/kmeans_composite_best.pkl")
    print("   Labels: results/kmeans_best_composite_labels.csv")

    # Show top 5 configs
    print("\n" + "="*80)
    print("📊 TOP 5 CONFIGS")
    print("="*80)
    # Chỉ chọn các cột có ý nghĩa với K-Means
    cols = ['n_clusters', 'init', 'n_init',
            'silhouette', 'calinski_harabasz', 'davies_bouldin']
    # Lọc các cột thực sự tồn tại trong df kết quả (đề phòng lỗi key)
    cols = [c for c in cols if c in df_results.columns]
    print(df_results[cols].head(5).to_string(index=False))

else:
    print("⚠️ No valid results found")

📂 Loading data for tuning from data/processed/encoded_data.csv...
  ✓ Data loaded: (3685, 58)

🚀 Starting Grid Search for: KMEANS
  Total combinations to test: 16


K-MEANS HYPERPARAMETER TUNING
Grid parameters:
  n_clusters: [3, 4, 5, 6, 7, 8, 9, 10]
  init: ['k-means++']
  n_init: [10, 20]
  max_iter: [300]

Total combinations: 16


  [1/16] KMEANS_n_clusters3_initk-means++_n_init10_max_iter300: Sil=0.2544, CH=1269.22, DB=1.5338, Clusters=3
  [2/16] KMEANS_n_clusters3_initk-means++_n_init20_max_iter300: Sil=0.2544, CH=1269.22, DB=1.5338, Clusters=3
  [3/16] KMEANS_n_clusters4_initk-means++_n_init10_max_iter300: Sil=0.2355, CH=1030.48, DB=1.8631, Clusters=4
  [4/16] KMEANS_n_clusters4_initk-means++_n_init20_max_iter300: Sil=0.2355, CH=1030.48, DB=1.8631, Clusters=4
  [5/16] KMEANS_n_clusters5_initk-means++_n_init10_max_iter300: Sil=0.2352, CH=894.63, DB=1.7419, Clusters=5
  [6/16] KMEANS_n_clusters5_initk-means++_n_init20_max_iter300: Sil=0.2352, CH=894.63, DB=1.7419, Clusters=5
  [7/16] KMEANS_n_clusters6_initk-means++_n_init10_max_iter300: Sil=0.1937, CH=820.04, DB=1.8492, Clusters=6
  [8/16] KMEANS_n_clusters6_initk-means++_n_init20_max_iter300: Sil=0.1938, CH=820.06, DB=1.8499, Clusters=6
  [9/16] KMEANS_n_clusters7_initk-means++_n_init10_max_iter300: Sil=0.2019, CH=777.00, DB=1.6383, Clusters=7
  [10/16] KMEAN

model_type  n_clusters  silhouette  calinski_harabasz  davies_bouldin  composite_score
    kmeans           3    0.254433        1269.217570        1.533831         0.701773
    kmeans           3    0.254433        1269.217570        1.533831         0.701773
    kmeans           7    0.201851         776.998807        1.638289         0.354408

🏆 BEST K-MEANS CONFIG
  n_clusters             : 3
  init                   : k-means++
  n_init                 : 10

  Silhouette Score       : 0.2544
  Calinski-Harabasz      : 1269.22
  Davies-Bouldin Index   : 1.5338

🔄 Training best model...


💾 Labels saved: results/kmeans_best_composite_labels.csv


✅ Best K-Means model saved!
   Model: results/kmeans_composite_best.pkl
   Labels: results/kmeans_best_composite_labels.csv

📊 TOP 5 CONFIGS
 n_clusters      init  n_init  silhouette  calinski_harabasz  davies_bouldin
          3 k-means++      10    0.254433        1269.217570        1.533831
          3 k-means++      20    0.254433        1269.217570        1.533831
          7 k-means++      20    0.201851         776.998807        1.638289
          7 k-means++      10    0.201851         776.998807        1.638289
          5 k-means++      20    0.235202         894.634671        1.741909
